In [2]:
import time
import re
import pytz
from pytz import timezone
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import string
from nltk import bigrams
from nltk import trigrams
import json
import pandas as pd
from unicodedata import normalize
 
punctuation = list(string.punctuation)
city_specific_stop_words =  ['York', 'Manhattan','NY','NewYork','NYC','The','A','#NewYork','#NYC','Brooklyn']
stop = stopwords.words('english') + punctuation + city_specific_stop_words + ['RT', 'rt', 'via', 'I' , 'like','\\',',',':','in','…','\ud83c','\ud83d','°','u2026','ud83d','ud83c']

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)


emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
        
    )"""


regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]


tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

count_all = Counter()

#twitter_dataset = pd.read_table('tweets_streamed_dataset.txt', index_col=None)
twitter_dataset = pd.read_table('data/nyc_streamed_dataset.txt', index_col=None)

for tweet in twitter_dataset.tweet_text:
    #tweet = normalize('NFKD', tweet).encode('ascii','ignore').decode('unicode_escape')
    tweet = normalize('NFKD', tweet)
    terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('#', '@')) and not emoji_pattern.search(term)]
    #terms_bigram = bigrams(terms_stop)
    terms_trigram = trigrams(terms_stop)
    #terms_hash = [term for term in preprocess(tweet1) if term.startswith('#')]
    #terms_all = [term for term in preprocess(tweet) if term not in stop]
    count_all.update(terms_trigram)
    #count_all.update(terms_hash)
    #count_all.update(terms_all)
    

print(count_all.most_common(100))
#twitter_dataset

[(('ufe', '0', 'f'), 338), (('n', 'n', 'n'), 308), (('Can', 'recommend', 'anyone'), 230), (('Just', 'posted', 'photo'), 209), (('Construction', 'Both', 'directions'), 170), (('could', 'great', 'fit'), 131), (('This', 'could', 'great'), 131), (('udd', '25', 'udd'), 124), (('25', 'udd', '25'), 123), (('This', 'might', 'great'), 119), (('might', 'great', 'fit'), 119), (('If', "you're", 'looking'), 113), (("you're", 'looking', 'work'), 113), (("We're", 'Read', 'latest'), 110), (('Read', 'latest', 'opening'), 110), (("We're", 'Click', 'apply'), 109), (('u2764', 'ufe', '0'), 107), (('View', 'latest', 'opening'), 104), (('team', 'See', 'latest'), 102), (('See', 'latest', 'opening'), 102), (('Interested', 'This', 'could'), 101), (("We're", 'Click', 'details'), 94), (('th', 'Street', 'Station'), 88), (('Update', 'Construction', 'Both'), 87), (('latest', 'click', 'apply'), 86), (('See', 'latest', 'click'), 86), (('Incident', 'WB', 'Exit'), 85), (('looking', 'work', 'check'), 84), (('work', 'View